# **Лабораторная работа №3**

### **Задание 1**.

Классификация

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

data = pd.read_csv("airlines_task.csv").drop_duplicates()
data
data.loc[:, ["Airline", "AirportFrom", "AirportTo", "Flight"]].nunique(axis=0)
data.dropna(inplace=True)
data_encoded = pd.get_dummies(data, columns=["Airline"], dtype=int).reset_index(drop=True)
routes = pd.Series(data=(data_encoded.AirportTo + data_encoded.AirportFrom), name="Route")
routes
data_encoded: pd.DataFrame = pd.concat([data_encoded, routes], axis=1).drop(["AirportTo", "AirportFrom"], axis=1)
data_encoded.rename({0: "Route"}, axis=1, inplace=True)
data_encoded.Route.value_counts()
label_map = dict(zip(data_encoded.Route.unique().tolist(), np.arange(0, data_encoded.Route.unique().shape[0])))
label_map
data_encoded.Route
data_encoded.Route = data_encoded.Route.map(lambda x: label_map.get(x))
data_encoded.Route

df = data_encoded

y = df['Delay']
X = df.drop(['Delay'], axis=1)

underSampler = RandomUnderSampler(sampling_strategy='majority')
X_under_sample, y_under_sample = underSampler.fit_resample(X, y)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_under_sample, y_under_sample, test_size=0.2, random_state = 4)

scaler = StandardScaler()

scaler.fit(X_train_clf, y_train_clf)
X_train_std_clf = scaler.transform(X_train_clf)
X_test_std_clf = scaler.transform(X_test_clf)

Поиск наилучших гиперпараметров

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

clf = DecisionTreeClassifier(criterion= best_params['criterion'],
                             splitter= best_params['splitter'],
                             max_depth= best_params['max_depth'],
                             min_samples_split= best_params['min_samples_split'],
                             min_samples_leaf= best_params['min_samples_leaf']
                             )
clf.fit(X_train_std_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_std_clf)
print(clf.score(X_test_std_clf, y_test_clf))

{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 2, 'splitter': 'best'}
0.8087813396532368


Метрики

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.8087813396532368
Confusion matrix:
 [[13864  6154]
 [ 1511 18556]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.90      0.69      0.78     20018
           1       0.75      0.92      0.83     20067

    accuracy                           0.81     40085
   macro avg       0.83      0.81      0.81     40085
weighted avg       0.83      0.81      0.81     40085



In [ ]:
from sklearn import tree

text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_0 <= 0.34
|   |--- feature_0 <= -0.92
|   |   |--- feature_0 <= -1.28
|   |   |   |--- feature_0 <= -1.99
|   |   |   |   |--- class: 0
|   |   |   |--- feature_0 >  -1.99
|   |   |   |   |--- class: 0
|   |   |--- feature_0 >  -1.28
|   |   |   |--- feature_20 <= 0.81
|   |   |   |   |--- feature_3 <= -0.81
|   |   |   |   |   |--- feature_3 <= -1.37
|   |   |   |   |   |   |--- feature_18 <= 2.07
|   |   |   |   |   |   |   |--- feature_3 <= -2.76
|   |   |   |   |   |   |   |   |--- feature_23 <= 0.67
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- feature_23 >  0.67
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- feature_3 >  -2.76
|   |   |   |   |   |   |   |   |--- feature_1 <= -0.93
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- feature_1 >  -0.93
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- feature_18 >  2.07
|   |   |

Регрессия

In [ ]:
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv("diamonds.csv", index_col=0).head(50000)
df = pd.get_dummies(df, columns=['cut','color','clarity'])
display(df)

y = df["price"]
X = df.drop(["price"], axis=1)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train_reg, y_train_reg)
X_train_reg = scaler.transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
1,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5,0.31,63.3,58.0,335,4.34,4.35,2.75,0,1,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,0.61,62.7,57.0,2192,5.43,5.36,3.38,0,0,0,...,0,0,0,0,0,0,1,0,0,0
49997,0.58,57.9,56.0,2192,5.52,5.49,3.19,1,0,0,...,0,0,0,0,0,0,1,0,0,0
49998,0.58,59.6,59.0,2192,5.43,5.38,3.22,0,0,0,...,0,0,0,0,0,0,0,1,0,0
49999,0.58,61.4,58.0,2192,5.37,5.34,3.29,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Поиск наилучших гиперпараметров

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


param_grid = {
    'criterion': ['squared_error', 'absolute_error'],
    'splitter': ['best'],
    'max_depth': [3, 4, 5]
}

grid_search = RandomizedSearchCV(DecisionTreeRegressor(), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

reg = DecisionTreeRegressor(criterion= best_params['criterion'],
                             splitter= best_params['splitter'],
                             max_depth= best_params['max_depth'])
reg.fit(X_train_reg, y_train_reg)
y_pred_reg = reg.predict(X_test_reg)
print(reg.score(X_test_reg, y_test_reg))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'splitter': 'best', 'max_depth': 5, 'criterion': 'squared_error'}
0.908715744097184


In [ ]:


print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 675.6956274014018
MSE: 1538029.0353247589
RMSE: 1240.1729860486232
MAPE: 0.4213225808127376
R^2: 0.908715744097184


In [ ]:
from sklearn import tree

text_representation = tree.export_text(reg)
print(text_representation)

|--- feature_0 <= 0.38
|   |--- feature_4 <= -0.14
|   |   |--- feature_0 <= -0.72
|   |   |   |--- feature_3 <= -0.96
|   |   |   |   |--- feature_3 <= -1.25
|   |   |   |   |   |--- value: [580.35]
|   |   |   |   |--- feature_3 >  -1.25
|   |   |   |   |   |--- value: [743.45]
|   |   |   |--- feature_3 >  -0.96
|   |   |   |   |--- feature_24 <= 1.71
|   |   |   |   |   |--- value: [926.38]
|   |   |   |   |--- feature_24 >  1.71
|   |   |   |   |   |--- value: [1180.81]
|   |   |--- feature_0 >  -0.72
|   |   |   |--- feature_21 <= 0.88
|   |   |   |   |--- feature_20 <= 0.62
|   |   |   |   |   |--- value: [1781.55]
|   |   |   |   |--- feature_20 >  0.62
|   |   |   |   |   |--- value: [1434.30]
|   |   |   |--- feature_21 >  0.88
|   |   |   |   |--- feature_0 <= -0.33
|   |   |   |   |   |--- value: [1188.83]
|   |   |   |   |--- feature_0 >  -0.33
|   |   |   |   |   |--- value: [1824.28]
|   |--- feature_4 >  -0.14
|   |   |--- feature_4 <= 0.23
|   |   |   |--- feature_21 <

### **Задание 2**


### BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier

base_estimator = DecisionTreeClassifier(random_state=0)

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0]
}

grid_search = GridSearchCV(BaggingClassifier(estimator=base_estimator,
                                random_state=0), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

{'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 30}
0.8071847324435575


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.8071847324435575
Confusion matrix:
 [[14758  5260]
 [ 2469 17598]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.86      0.74      0.79     20018
           1       0.77      0.88      0.82     20067

    accuracy                           0.81     40085
   macro avg       0.81      0.81      0.81     40085
weighted avg       0.81      0.81      0.81     40085



### BaggingRegressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

base_estimator = DecisionTreeRegressor(random_state=0)

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0]
}

grid_search = GridSearchCV(BaggingRegressor(estimator=base_estimator,
                               random_state=0), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

{'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 30}
0.9804773359884774


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 289.4050970555556
MSE: 328933.21854513994
RMSE: 573.5269989679125
MAPE: 0.2613272164969985
R^2: 0.9804773359884774


### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(GradientBoostingClassifier(random_state=0), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

{'n_estimators': 300}
0.815517026319072


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.815517026319072
Confusion matrix:
 [[13811  6207]
 [ 1188 18879]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.92      0.69      0.79     20018
           1       0.75      0.94      0.84     20067

    accuracy                           0.82     40085
   macro avg       0.84      0.82      0.81     40085
weighted avg       0.84      0.82      0.81     40085



### GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.0001]
}

grid_search = RandomizedSearchCV(GradientBoostingRegressor(random_state=0), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 300, 'learning_rate': 0.0001}
0.050693277644815016


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 3062.112649844802
MSE: 15994667.294715967
RMSE: 3999.333356287766
MAPE: 1.3869030900251567
R^2: 0.050693277644815016


### StackingClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

base_estimators = [('svc', SVC(probability=True)), ('dt', DecisionTreeClassifier())]

final_estimator = LogisticRegression()

clf = StackingClassifier(estimators=base_estimators, final_estimator=final_estimator)
clf.fit(X_train_std_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_std_clf)
print(clf.score(X_test_std_clf, y_test_clf))

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

### StackingRegressor

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

base_estimators = [('svr', SVR()), ('dt', DecisionTreeRegressor())]

final_estimator = LinearRegression()

reg = StackingRegressor(estimators=base_estimators, final_estimator=final_estimator)
reg.fit(X_train_reg, y_train_reg)

y_pred_reg = reg.predict(X_test_reg)
print(reg.score(X_test_reg, y_test_reg))

In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

### Задание 3.
### Попробовать для решения задачи регрессии и классификации одну из библиотек (на выбор)

### LightGBM

### Классификация

In [ ]:
from lightgbm import LGBMClassifier


param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}


grid_search = GridSearchCV(LGBMClassifier(), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)


y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

### Регрессия

In [ ]:
from lightgbm import LGBMRegressor


param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}


grid_search = GridSearchCV(LGBMRegressor(), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)


y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

### Задание 4.
### Реализовать обучения и вычисления дерева с использованием алгоритма CART для задачи регрессии и классификации.

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [ ]:
class CART():
    def __init__(self, min_samples_split=2, max_depth=2, isRegression = False):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.isRegression = isRegression

    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        if self.mode == "entropy":
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        else:
            gain = self.mean_squared_error(parent) - (weight_l*self.mean_squared_error(l_child) + weight_r*self.mean_squared_error(r_child))
        return gain

    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def mean_squared_error(self, y):
        mean = np.mean(y)
        mse = np.mean((y - mean)**2)
        return mse

    def calculate_leaf_value(self, Y):
        if self.isRegression:
            return np.mean(Y)
        Y = list(Y)
        return max(Y, key=Y.count)

    def fit(self, X, Y):
        if self.isRegression:
            self.mode = "mse"
        else:
            self.mode = "entropy"
        dataset = np.column_stack((X, Y))
        self.root = self.build_tree(dataset)

    def predict(self, X):
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)